# AI 201 Programming Assignment 1
## A* Algorithm Implementation

Submitted by: 
Jan Lendl R. Uy, 2019-00312

In [1]:
# CONSTANTS

# Relative path to the input file
INPUT_FILE_PATH = "astar_in.txt"

# Set a maximum number of iterations for A* to avoid infinite loop
MAX_A_STAR_ITERATIONS = 50000

## File Handling
This function reads an input .txt file containing the start and goal states of the puzzle then converts the puzzle contents into a two-dimensional tuple of tile values. The input file is stored in the same directory as the notebook file with the filename of "astar_in.txt".

In [2]:
def read_file(path):
    
    with open(path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store the start state and goal state
    start_state = []
    goal_state = []

    # Initialize a flag to keep track if following tiles belong to the start
    # or the goal state
    reading_part = None

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace
        if line == "start":
            reading_part = "start"
        elif line == "goal":
            reading_part = "goal"
        elif line and reading_part:
            # Convert line into a list, handling "*" and converting numbers to integers
            line_list = [int(x) if x.isdigit() else x for x in line.split()]
            if reading_part == "start":
                start_state.append(line_list)
            elif reading_part == "goal":
                goal_state.append(line_list)
                
    start_state = tuple(tuple(row) for row in start_state)
    goal_state = tuple(tuple(row) for row in goal_state)

    return start_state, goal_state

In [3]:
start_state, goal_state = read_file(INPUT_FILE_PATH)
print(f"Start State: {start_state}")
print(f"Goal State: {goal_state}")

Start State: ((2, 1, 6), (4, '*', 8), (7, 5, 3))
Goal State: ((1, 2, 3), (8, '*', 4), (7, 6, 5))


## PuzzleBoard
This class encapsulates the puzzle as a tuple of tuples (i.e. 2D tuple).

Among the operators overridden in this class are the `__str__` method to "prettify" the printing of the 8-puzzle and the `__eq__` and `__hash__` for checking of equal board contents. Aside from this, the following methods were implemented to simplify the operations of the game.

In [4]:
class PuzzleBoard:
    
    def __init__(self, board):
        self.board = board
        
        # Assume that the puzzle is a square
        self.rows = len(board)
        self.cols = len(board[0])
        
        self.board_as_string = self.convert_board_to_string()
        
    def convert_board_to_string(self):
        if not self.board:
            return "Empty board"
        
        board = ""

        for i in range(self.rows):
            board += "| "
            for j in range(self.cols):
                tile = self.board[i][j]
                board += str(tile)
                board += " | "
            board += "\n"
            
        return board

    def __str__(self):
                    
        return self.board_as_string
    
    def __eq__(self, other):
        if not isinstance(other, PuzzleBoard):
            return False
        return self.board == other.board

    def __hash__(self):
        return hash(self.board)
    
    def _find_empty_tile(self):
        for i, row in enumerate(self.board):
            for j, tile in enumerate(row):
                if tile == "*":
                    return (i, j)
        raise ValueError("No empty tile found in the puzzle")
    
    def get_possible_moves(self):
        empty_row, empty_col = self._find_empty_tile()
        possible_moves = []
        
        # Possible moves: up, down, left, right
        # Tuples indicate the numbers added to the coordinates of the empty tile
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for direction in directions:
            new_row = empty_row + direction[0]
            new_col = empty_col + direction[1]
            
            # Check if the new position is within the board
            if 0 <= new_row < len(self.board) and 0 <= new_col < len(self.board[0]):
                # Create a new board configuration with the move applied
                new_board = [list(row) for row in self.board]  # Convert tuple of tuples to list of lists
                new_board[empty_row][empty_col], new_board[new_row][new_col] = new_board[new_row][new_col], new_board[empty_row][empty_col]
                new_board = tuple(tuple(row) for row in new_board)  # Convert back to tuple of tuples
                
                possible_moves.append(new_board)
        
        return possible_moves
    
    # Helper methods useful for computing the different heuristic functions h(n)
    
    def get_misplaced_tiles(self, goal_puzzle):
        misplaced = 0
        goal_state = goal_puzzle.board
        for i in range(self.rows):
            for j in range(self.cols):
                if self.board[i][j] != "*" and self.board[i][j] != goal_state[i][j]:
                    misplaced += 1
        return misplaced
    
    def get_manhattan_distance(self, goal_puzzle):
        total_distance = 0
        goal_state = goal_puzzle.board

        for i in range(self.rows):
            for j in range(self.cols):
                if self.board[i][j] != "*":
                    # Find the tile's position in the goal state
                    for gi in range(self.rows):
                        for gj in range(self.cols):
                            if self.board[i][j] == goal_state[gi][gj]:
                                # Calculate Manhattan distance for this tile
                                distance = abs(i - gi) + abs(j - gj)
                                total_distance += distance
                                break
                        else:
                            continue
                        break

        return total_distance
    
    def get_nilsson_sequence_score(self, goal_puzzle):
        current_board = self.board
        goal_board = goal_puzzle.board

        manhattan_dist = self.get_manhattan_distance(goal_puzzle)
        
        # Flatten the boards for easier index computation
        current_board_flat = [tile for row in current_board for tile in row]
        goal_board_flat = [tile for row in goal_board for tile in row]
        
        # The correct clockwise ordering of the tiles around the periphery
        goal_order = goal_board_flat[0:3] + [goal_board_flat[5], goal_board_flat[8], goal_board_flat[7], goal_board_flat[6], goal_board_flat[3]]
        current_order = current_board_flat[0:3] + [current_board_flat[5], current_board_flat[8], current_board_flat[7], current_board_flat[6], current_board_flat[3]]

        # Calculate sequence score
        goal_pairs = []
        current_pairs = []
        sequence_score = 0
        for i in range(-1, -len(goal_order), -1):
            current_pair = (current_order[i], current_order[i+1])
            goal_pair = (goal_order[i], goal_order[i+1])
            goal_pairs.append(goal_pair)
            if current_pair[0] == "*":
                continue
            current_pairs.append(current_pair)

        for pair in current_pairs:
            if pair not in goal_pairs:
                sequence_score += 2

        # Check the center square, add 1 if it is not the empty square
        if current_board_flat[4] != "*":
            sequence_score += 1
        
        # Multiply the sequence score by 3 and add the Manhattan distance
        return 3 * sequence_score + manhattan_dist

In [5]:
start_puzzle = PuzzleBoard(start_state)
goal_puzzle = PuzzleBoard(goal_state)
print(f"Start Puzzle: \n{start_puzzle}")
print(f"Goal Puzzle: \n{goal_puzzle}")

Start Puzzle: 
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Goal Puzzle: 
| 1 | 2 | 3 | 
| 8 | * | 4 | 
| 7 | 6 | 5 | 



In [6]:
class PuzzleSearchNode:

    def __init__(self, puzzle, heuristic, g=0, parent=None):
        self.puzzle = puzzle
        self.parent = parent
        self.heuristic = heuristic
        self.g = g  # Path cost from start node to current node
        self.h = 0  # Heuristic estimate from current node to goal
        self.f = 0  # Total estimated path cost (g + h)

    def expand(self):
        expanded = []
        for new_puzzle in self.puzzle.get_possible_moves():
            # Increment path cost of child node by 1
            child_path_cost = self.g + 1
            child_puzzle = PuzzleBoard(new_puzzle)
            child = PuzzleSearchNode(child_puzzle, self.heuristic, child_path_cost, self)
            expanded.append(child)
        return expanded

    def compute_costs(self, goal_node):
        goal_puzzle = goal_node.puzzle
        if self.heuristic == "misplaced":
            self.h = self.puzzle.get_misplaced_tiles(goal_puzzle)
        elif self.heuristic == "manhattan":
            self.h = self.puzzle.get_manhattan_distance(goal_puzzle)
        elif self.heuristic == "nilsson":
            self.h = self.puzzle.get_nilsson_sequence_score(goal_puzzle)
        self.f = self.g + self.h

    def goal_test(self, goal_node):
        return self == goal_node

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return isinstance(other, PuzzleSearchNode) and self.puzzle == other.puzzle

    def __hash__(self):
        return hash(self.puzzle)

In [7]:
def astar_search(start_puzzle, goal_puzzle, heuristic="misplaced"):
    
    start_node = PuzzleSearchNode(start_puzzle, heuristic)
    goal_node = PuzzleSearchNode(goal_puzzle, heuristic)
    
    # Compute initial costs for the start node
    start_node.compute_costs(goal_node)
    
    # Initialize open and closed lists
    open_list = [start_node]
    closed_list = set()
    
    search_cost = 0

    print(f"Initial estimated path cost f = {start_node.f}")
    
    for i in range(MAX_A_STAR_ITERATIONS):
        if not open_list:
            return "Failed to find the goal node!"
        
        # Find the node with the lowest f value
        current_node = min(open_list)
                
        # If the current node is the goal node, return the goal node
        if current_node.goal_test(goal_node):
            print("Goal node found!")
            return i, search_cost, current_node
        
        # Move the current node from the open list to the closed list
        open_list.remove(current_node)
        closed_list.add(current_node)
        
        expansion = current_node.expand()
        search_cost += len(expansion)
                
        # Expand the current node
        for child_node in expansion:
            # If the child is already in closed_list, skip this node
            if child_node in closed_list:
                continue
            
            # Compute costs for the child node
            child_node.compute_costs(goal_node)
            
            # If the child is not in open_list, insert this to the list
            if child_node not in open_list:
                open_list.append(child_node)
            else:
                # If the child is already in open_list, check if this path is better
                existing_node = open_list[open_list.index(child_node)]
                if child_node.g < existing_node.g:
                    # This path is better, so update the node
                    existing_node.g = child_node.g
                    existing_node.f = child_node.f
                    existing_node.parent = current_node
        
        # print(f"""
        #       Iteration {i+1}:
        #         Current f = {current_node.f}
        #         Open list size: {len(open_list)}
        #         Closed list size: {len(closed_list)}
        #       """
        # )
        
    return f"Failed to find a solution within {MAX_A_STAR_ITERATIONS} iterations!"

def get_solution(goal_node):
    solution = [goal_node]
    current_node = goal_node
    while True:
        current_node = current_node.parent
        if current_node == None:
            break
        solution.append(current_node)
    solution.reverse()
    return solution

In [8]:
iters_misplaced, search_cost_misplaced, result_misplaced = astar_search(start_puzzle, goal_puzzle)
print(f"Solution found for {iters_misplaced} iterations of A* search. Heuristic function is the number of misplaced tiles.")

Initial estimated path cost f = 7
Goal node found!
Solution found for 1746 iterations of A* search. Heuristic function is the number of misplaced tiles.


In [9]:
iters_manhattan, search_cost_manhattan, result_manhattan = astar_search(start_puzzle, goal_puzzle, "manhattan")
print(f"Solution found for {iters_manhattan} iterations of A* search. Heuristic function is Manhattan Distance.")

Initial estimated path cost f = 12
Goal node found!
Solution found for 226 iterations of A* search. Heuristic function is Manhattan Distance.


In [10]:
iters_nilsson, search_cost_nilsson, result_nilsson = astar_search(start_puzzle, goal_puzzle, "nilsson")
print(f"Solution found for {iters_nilsson} iterations of A* search. Heuristic function is Nilsson Sequence Score.")

Initial estimated path cost f = 54
Goal node found!
Solution found for 30 iterations of A* search. Heuristic function is Nilsson Sequence Score.


## Sanity Checking
Ensure that the solution by A* for each heuristic function is EXACTLY the same.

In [11]:
solution_astar_misplaced = get_solution(result_misplaced)
solution_astar_manhattan = get_solution(result_manhattan)
solution_astar_nilsson = get_solution(result_nilsson)

for i in range(len(solution_astar_misplaced)):
    is_equal = solution_astar_misplaced[i] == solution_astar_manhattan[i] == solution_astar_nilsson[i]
    if not is_equal:
        print(f"There is an inconsistent solution among the three runs of A*")
        break